In [1]:
import geemap
import ee

Map = geemap.Map()
Map

Map(center=[0, 0], controls=(WidgetControl(options=['position', 'transparent_bg'], widget=SearchDataGUI(childr…

In [3]:
Map.add_basemap("")

In [ ]:
import os
import geopandas as gpd

os.chdir(r'D:\Work\Problem')

shp_file = r'JX.shp'

In [ ]:
shp_gdf = gpd.read_file(shp_file)
shp = geemap.gdf_to_ee(shp_gdf)

roi = shp.geometry()
Map.addLayer(roi, {}, 'roi')
Map.center_object(roi, 7)

In [ ]:
star_time = "2015"
end_time = "2021"

In [ ]:
def cloudMaskL457(image):
    qaMask = image.select('QA_PIXEL').bitwiseAnd(int('11111', 2)).eq(0)
    saturationMask = image.select('QA_RADSAT').eq(0)
    opticalBands = image.select('SR_B.').multiply(0.0000275).add(-0.2)

    return image.addBands(opticalBands, None, True)\
                 .updateMask(qaMask).updateMask(saturationMask)
                    
def maskS2clouds(image): 
    qa = image.select('QA60')


    cloudBitMask = 1 << 10
    cirrusBitMask = 1 << 11

    mask = qa.bitwiseAnd(cloudBitMask).eq(0).And(
             qa.bitwiseAnd(cirrusBitMask).eq(0))

    return image.updateMask(mask).copyProperties(image, ["system:time_start"])

In [ ]:
def calc_landsat_ndvi(image):
    ndvi = image.normalizedDifference(['SR_B5','SR_B4']).rename('NDVI')
    return ndvi

def calc_s2_ndvi(image):
    ndvi = image.normalizedDifference(['B8','B4']).rename('NDVI')
    return ndvi

In [ ]:
# 获取研究区landsat8影像数据, 并进行去云处理
images = ee.ImageCollection("LANDSAT/LC08/C02/T1_L2").filterBounds(roi).filterDate(star_time, end_time)\
                    .filter(ee.Filter.lt('CLOUD_COVER',30)).map(cloudMaskL457)

In [ ]:
# 计算ndvi
ndvis = images.map(calc_landsat_ndvi)

# 获取每个像元时间序列上ndvi最值
ndvi_max = ndvis.max()                    
ndvi_min = ndvis.min()

## **VFC计算**
### $VFC = (NDVI - NDVI_{Soi}) / (NDVI_{Veg} - NDVI_{Soi})$
##### **NDVISoil**: 裸土或无植被区的NDVI值,这里使用NDVI最小值影像百分位处于5%的值作为替代
##### **NDVIVeg**: 为纯植被区的NDVI值, 这里使用NDVI最大值影像百分位处于95%的值作为替代

In [ ]:
xmax = ndvi_max.reduceRegion(**{
    'geometry': roi,
    'reducer': ee.Reducer.percentile([95]),
    'scale': 30,
    'maxPixels': 1e13
}).get("NDVI")

xmin = ndvi_min.reduceRegion(**{
    'geometry': roi,
    'reducer': ee.Reducer.percentile([5]),
    'scale': 30,
    'maxPixels': 1e13
}).get("NDVI")

#### 计算后将xmax和xmin导入到asset中

In [ ]:
ndvi_max_min = ee.FeatureCollection([ee.Feature(roi, {"xmax": ee.Number(xmax), "xmin": ee.Number(xmin)})])
geemap.ee_export_vector_to_asset(ndvi_max_min, "ndvi_max_min", "ndvi_max_min")

#### 重新从asset导入xmax和xmin

In [ ]:
ndvi_max_min_fc = ee.FeatureCollection("users/2431566134liumonarch/ndvi_max_min")
xmax = ndvi_max_min_fc.first().get("xmax")
xmin = ndvi_max_min_fc.first().get("xmin")

In [ ]:
def calc_vfc(image):
    fv = image.expression('(ndvi - xmin) / (xmax - xmin)',{
        'ndvi': image.select("NDVI"),
        'xmin': ee.Number(xmin),
        'xmax': ee.Number(xmax)
        }).rename('vfc')
    return fv

In [ ]:
vfcs = ndvis.map(calc_vfc)

palette = ['FFFFFF', 'CE7E45', 'DF923D', 'F1B555', 'FCD163', '99B718',
               '74A901', '66A000', '529400', '3E8601', '207401', '056201',
               '004C00', '023B01', '012E01', '011D01', '011301']

In [ ]:
Map.addLayer(vfcs.mosaic().clip(roi), {"min":0, "max":1, "palette":palette}, "vfc")